In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.figsize']=(20,5)
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pyarrow.parquet as pq

In [2]:
def sliderPlot(df1,i):
    fig = px.line(df1, y=df1[i] )
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ))
    fig.show()

In [3]:
table = pq.read_table(r'C:/Users/Harsh/Downloads/DLMS Data/data.parquet')

# Convert it to a Pandas DataFrame (optional)
dframe = table.to_pandas()

In [4]:
dframe.head(1)
dframe['Kwh']=dframe['Kwh']/1000

In [5]:
id=dframe['sensor'].unique()
id

array(['5f718c439c7a78.65267835', '62a9920f75c931.62399458'], dtype=object)

In [6]:
lst=[]
for location in id:
    df=dframe[dframe['sensor'] == location ]
    # df['Clock']=pd.to_datetime(df['Clock'])
    df.set_index(['Clock'],inplace=True,drop= True)
    df = df[df.index >= '2022-11-18 00:00:00']
    filtered_df = df[((df['R_Voltage'] == 0) | (df['Y_Voltage'] == 0) | (df['B_Voltage'] == 0)) & 
                     ((df['R_Current'] == 0) | (df['Y_Current'] == 0) | (df['B_Current'] == 0))]
    filtered_df['Kwh'] = 0
    df.loc[filtered_df.index, :] = filtered_df
    df=df[['Kwh']].resample(rule='1H').sum()
    # df['id']=location
    df['Kwh_r']=df['Kwh'].rolling(window=24).mean()
    df.dropna(inplace=True)

    lst.append(df)

In [7]:
df=lst[0]
df

,Kwh,Kwh_r
Clock,,
2022-11-18 23:00:00,665.2,717.695833
2022-11-19 00:00:00,592.3,731.487500
2022-11-19 01:00:00,526.3,732.204167
2022-11-19 02:00:00,505.7,733.212500
2022-11-19 03:00:00,487.4,733.845833
...,...,...
2023-10-31 22:00:00,1066.9,985.366667
2023-10-31 23:00:00,975.5,983.391667
2023-11-01 00:00:00,887.0,981.958333


In [8]:
df['Kwh'].describe()

count    8332.000000
mean     1231.005713
std       502.267511
min         0.000000
25%       854.175000
50%      1180.800000
75%      1535.750000
max      3397.200000
Name: Kwh, dtype: float64

In [9]:
result=adfuller(df['Kwh'],autolag='AIC')
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

ADF Statistic: -3.0948425262201607
p-value: 0.026937226462572607
Critical Values: {'1%': -3.4311382062666165, '5%': -2.8618883324211724, '10%': -2.56695541251651}
